In [42]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import json
import tweepy
import time
import seaborn as sn

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [4]:
consumer_key = "K7iJKXOFsPP8ecFS5y3OcYFfS"
consumer_secret = "FFSC2Y2ODnEyMPWxcnSvB5OZ06w3l1Ffb4xGyTecxlkmMRuUnq"
access_token = "975022286091800582-Ywq9TMjelr5WxavjEPmdBsBeuVM7dW0"
access_token_secret = "5PAjFT27QMrxfnAJH1vZfG0NA2owqRALMYgvSPSNPo5TA"

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [6]:
news_site = ('@BBC', '@CBS', '@CNN', '@Fox', '@New York Times')

In [32]:
tweet_times = []
user_list = []
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
time_list = []
my_index = []

for site in news_site:
    idx = 0
    for x in range(5):
        public_tweets = api.user_timeline(site)
        
        # Loop through tweets
        for tweet in public_tweets:

            # Run Vader Analysis on each tweet
            compound = analyzer.polarity_scores(tweet["text"])["compound"]
            pos = analyzer.polarity_scores(tweet["text"])["pos"]
            neu = analyzer.polarity_scores(tweet["text"])["neu"]
            neg = analyzer.polarity_scores(tweet["text"])["neg"]
            

            user_list.append(site)
            compound_list.append(compound)
            positive_list.append(pos)
            negative_list.append(neg)
            neutral_list.append(neu)
            my_index.append(idx)
            idx +=1

df1 = pd.DataFrame({'Agency': user_list, 'Compound': compound_list,'Positive': positive_list,
                            'Neutral':neutral_list,'Negative':negative_list,
                         'My_index':my_index
                         })


In [45]:
df1.set_index('My_index').head()

,Agency,Compound,Negative,Neutral,Positive
My_index,,,,,
0,@BBC,0.4404,0.088,0.737,0.175
1,@BBC,0.3919,0.124,0.725,0.151
2,@BBC,-0.3612,0.185,0.815,0.000
3,@BBC,-0.4588,0.111,0.845,0.044
4,@BBC,-0.2263,0.101,0.899,0.000


In [34]:
df1['Agency'].unique()

array(['@BBC', '@CBS', '@CNN', '@Fox', '@New York Times'], dtype=object)

In [46]:
plt.figure(figsize=(12,8))
data = df1[df1['Agency'] == '@Fox'].index
plt.scatter(data, df1[df1['Agency']=='@Fox']['Compound'],  marker ='o', color='b', s=60, label = 'FOX')
plt.scatter(data,df1[df1['Agency']=='@CNN']['Compound'],  marker ='o', color='y', s=60, label = 'CNN')
plt.scatter(data, df1[df1['Agency']=='@New York Times']['Compound'], marker ='o', s=60, color='m', label = 'NYT')
plt.scatter(data,df1[df1['Agency']=='@BBC']['Compound'], marker ='o', s = 60, color='g', label = 'BBC')
plt.scatter(data,df1[df1['Agency']=='@CBS']['Compound'], marker ='o', color='r', label = 'CBS')
plt.gca().set(xlabel = 'Tweets Ago', ylabel = 'Tweet Polarity',title = 'Sentiment Analysis of Media Tweets')
plt.legend(loc = 'best',bbox_to_anchor=(1, 0.5), frameon=True)

plt.show()

ValueError: x and y must be the same size

In [47]:
Compound_percentage =  df1.groupby('Agency').mean()['Compound'].to_frame("% Compound")
Compound_percentage

,% Compound
Agency,
@BBC,0.037075
@CBS,0.419605
@CNN,-0.077595
@Fox,0.182335
@New York Times,0.000000


In [50]:
x=Compound_percentage.index
y=Compound_percentage['% Compound']
plt.figure(figsize=(16,8))
colors = ['green' if _y >=0.0 else 'red' for _y in y]
ax = sn.barplot(x, y, palette=colors)

plt.gca().set(xlabel='News Channels', ylabel='% Tweet Polarity', title='Overall Media Sentiment')
plt.rc('grid', linestyle="--", color='black', linewidth=0.5)
plt.grid(True)
plt.plot()
plt.show()